# Clean-up your SageMaker environment

❗ This notebook removes all resources created by all notebooks you run in your environment. The following code cells will:
- permanently delete project or projects you provisioned in your Studio environment
- permanently delete feature group or groups
- permanently delete project-provisioned S3 buckets
- permanently delete objects in S3 buckets under project-related prefixes
- permanently delete inference endpoints

<div class="alert alert-info"> 💡 <strong> This notebook will permanently delete resources in your AWS account. Please double check the names of resources to be deleted! </strong>
</div>

In [2]:
import sagemaker
import boto3
import time
import json
import os

In [3]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-start-here notebook   ")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")

Stored variables and their in-db values:
baseline_s3_url                        -> 's3://sagemaker-us-east-1-462832133259/from-idea-t
bucket_name                            -> 'sagemaker-us-east-1-462832133259'
bucket_prefix                          -> 'from-idea-to-prod/xgboost'
domain_id                              -> 'd-1tywfpuzdgqt'
evaluation_s3_url                      -> 's3://sagemaker-us-east-1-462832133259/from-idea-t
initialized                            -> True
input_s3_url                           -> 's3://sagemaker-us-east-1-462832133259/from-idea-t
model_package_group_name               -> 'from-idea-to-prod-model-group'
output_s3_url                          -> 's3://sagemaker-us-east-1-462832133259/from-idea-t
prediction_baseline_s3_url             -> 's3://sagemaker-us-east-1-462832133259/from-idea-t
region                                 -> 'us-east-1'
sm_role                                -> 'arn:aws:iam::462832133259:role/cfnstudiodomain-Sa
test_s3_url         

In [4]:
sm = boto3.client("sagemaker")
s3 = boto3.resource('s3')

## Delete projects

In [5]:
# Get all projects created by the current domain
projects = [
    {"ProjectName":p["ProjectName"], "ProjectId":p["ProjectId"]} for p in sm.list_projects(MaxResults=100, SortBy="CreationTime")["ProjectSummaryList"] 
        if sm.describe_project(ProjectName=p["ProjectName"])["CreatedBy"]["DomainId"] == domain_id and p["ProjectStatus"] == "CreateCompleted"
]

print(f"These projects have been created by domain {domain_id}: {json.dumps(projects, indent=2)}")

These projects have been created by domain d-1tywfpuzdgqt: [
  {
    "ProjectName": "model-deploy-18-09-00-37",
    "ProjectId": "p-bosxs1pdacsx"
  },
  {
    "ProjectName": "model-build-18-07-54-29",
    "ProjectId": "p-xa5gxbybcnsa"
  }
]


In [6]:
# Select projects to be deleted
projects_to_delete = []

for p in projects:
    print(f"Are you sure you want to delete this project: {p['ProjectName']}? (y/n)")
    choice = input()
    if choice == 'y':
        projects_to_delete.append(p)
        
print(f"***************************************")
print(f"The following projects will be deleted:\n{json.dumps(projects_to_delete, indent=2)}")
print(f"***************************************")

Are you sure you want to delete this project: model-deploy-18-09-00-37? (y/n)


 y


Are you sure you want to delete this project: model-build-18-07-54-29? (y/n)


 y


***************************************
The following projects will be deleted:
[
  {
    "ProjectName": "model-deploy-18-09-00-37",
    "ProjectId": "p-bosxs1pdacsx"
  },
  {
    "ProjectName": "model-build-18-07-54-29",
    "ProjectId": "p-xa5gxbybcnsa"
  }
]
***************************************


In [7]:
for p in projects_to_delete:
    try:
        print(f"Deleting project {p['ProjectName']}:{sm.delete_project(ProjectName=p['ProjectName'])}")
    except Exception:
        pass

Deleting project model-deploy-18-09-00-37:{'ResponseMetadata': {'RequestId': '68b629c7-df47-425a-be8d-f51835821b7e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '68b629c7-df47-425a-be8d-f51835821b7e', 'content-type': 'application/x-amz-json-1.1', 'content-length': '0', 'date': 'Sat, 18 Feb 2023 10:03:52 GMT'}, 'RetryAttempts': 0}}
Deleting project model-build-18-07-54-29:{'ResponseMetadata': {'RequestId': '3b682faa-939d-45e9-992b-9e0d654944bc', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '3b682faa-939d-45e9-992b-9e0d654944bc', 'content-type': 'application/x-amz-json-1.1', 'content-length': '0', 'date': 'Sat, 18 Feb 2023 10:03:55 GMT'}, 'RetryAttempts': 2}}


## Delete monitoring schedules
You must delete monitoring schedules to be able to successfully remove endpoints deployed by the MLOps projects.

In [8]:
def delete_mon_schedules(endpoint_name, sm_client):
    print(f"Delete monitoring schedules for the endpoint: {endpoint_name}")
    for s in sm_client.list_monitoring_schedules(EndpointName=endpoint_name)["MonitoringScheduleSummaries"]:
        print(f"Delete monitoring schedule: {s['MonitoringScheduleName']}")
        r = sm_client.delete_monitoring_schedule(MonitoringScheduleName=s['MonitoringScheduleName'])
        print(r)

In [9]:
for p in projects_to_delete:
    delete_mon_schedules(f"{p['ProjectName']}-staging", sm)
    delete_mon_schedules(f"{p['ProjectName']}-prod", sm)

Delete monitoring schedules for the endpoint: model-deploy-18-09-00-37-staging
Delete monitoring schedules for the endpoint: model-deploy-18-09-00-37-prod
Delete monitoring schedules for the endpoint: model-build-18-07-54-29-staging
Delete monitoring schedules for the endpoint: model-build-18-07-54-29-prod


## Delete CloudFormation stacks
This section deletes the AWS CloudFormation stacks that projects created

In [ ]:
cfn = boto3.client("cloudformation")

for p in projects_to_delete:
    for s in [
            f"sagemaker-{p['ProjectName']}-{p['ProjectId']}-deploy-staging",
            f"sagemaker-{p['ProjectName']}-{p['ProjectId']}-deploy-prod"
            ]:
        try:
            print(f"Delete CloudFormation stack: {s}")
            r = cfn.delete_stack(StackName=s)
            print(r)
            time.sleep(180)
        except Exception as e:
            print(f"Exception in deleting {s}:{e}")
            pass

Delete CloudFormation stack: sagemaker-model-deploy-18-09-00-37-p-bosxs1pdacsx-deploy-staging
{'ResponseMetadata': {'RequestId': '3b3b1f3b-5a8c-4db9-af8b-1abea700016c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '3b3b1f3b-5a8c-4db9-af8b-1abea700016c', 'date': 'Sat, 18 Feb 2023 10:04:37 GMT', 'content-type': 'text/xml', 'content-length': '212'}, 'RetryAttempts': 0}}
Delete CloudFormation stack: sagemaker-model-deploy-18-09-00-37-p-bosxs1pdacsx-deploy-prod
{'ResponseMetadata': {'RequestId': '52f013df-df22-4192-9e18-0d78294f70b5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '52f013df-df22-4192-9e18-0d78294f70b5', 'date': 'Sat, 18 Feb 2023 10:07:37 GMT', 'content-type': 'text/xml', 'content-length': '212'}, 'RetryAttempts': 0}}
Delete CloudFormation stack: sagemaker-model-build-18-07-54-29-p-xa5gxbybcnsa-deploy-staging
{'ResponseMetadata': {'RequestId': '4221c0da-2584-47a4-a14d-62c96712b468', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4221c0d

## Delete feature groups

In [12]:
feature_groups = sm.list_feature_groups(
    FeatureGroupStatusEquals="Created", 
    SortOrder="Descending", 
    SortBy="CreationTime"
)["FeatureGroupSummaries"]

In [13]:
feature_groups

[]

In [14]:
# Select feature groups to be deleted
feature_groups_to_delete = []

for fg in feature_groups:
    print(f"Are you sure you want to delete this feature group: {fg['FeatureGroupName']}? (y/n)")
    choice = input()
    if choice == 'y':
        feature_groups_to_delete.append(fg["FeatureGroupName"])
        
print(f"********************************************")
print(f"The following feature groups will be deleted:\n{json.dumps(feature_groups_to_delete, indent=2)}")
print(f"********************************************")

********************************************
The following feature groups will be deleted:
[]
********************************************


In [15]:
def delete_offline_store(feature_group_name: str):
    try:
        offline_store_config = sm.describe_feature_group(FeatureGroupName=feature_group_name)['OfflineStoreConfig']

    except Exception:
        print(f'Feature group: {feature_group_name} does NOT have an offline store!')
        return
    
    offline_store_s3_uri = offline_store_config['S3StorageConfig']['ResolvedOutputS3Uri']
    print(f"all feature store objects under {offline_store_s3_uri} will be deleted!")
    print("Are you sure you want to these objects ? (y/n)")
    
    choice = input()
    if choice == 'y':
        !aws s3 rm {offline_store_s3_uri} --recursive

<div class="alert alert-info"> 💡 <strong> The following code cell will delete the selected feature groups!</strong>
</div>

In [16]:
for fg in feature_groups_to_delete:
    print(f"Deleting the feature group: {fg}")
    delete_offline_store(fg)
    sm.delete_feature_group(FeatureGroupName=fg)

## Delete project-provisioned S3 buckets

<div class="alert alert-info"> 💡 <strong> The following code cell will delete all S3 buckets created by a project!</strong> If you are in an instructor-led workshop and using a provisioned AWS account, you might be not able to delete any S3 buckets. You can stop here.
</div>

In [17]:
print(f"*****************************************************")
print(f"The following S3 buckets will be removed permanently!")
print(f"*****************************************************")
for p in projects_to_delete:
    print(f"sagemaker-project-{p['ProjectId']}")

*****************************************************
The following S3 buckets will be removed permanently!
*****************************************************
sagemaker-project-p-bosxs1pdacsx
sagemaker-project-p-xa5gxbybcnsa


In [18]:
for p in projects_to_delete:
    !aws s3 rb s3://sagemaker-project-{p['ProjectId']} --force 

delete: s3://sagemaker-project-p-bosxs1pdacsx/sagemaker-model-depl/BuildArtif/bl9AbQl
delete: s3://sagemaker-project-p-bosxs1pdacsx/sagemaker-model-depl/BuildArtif/DVyER9v
delete: s3://sagemaker-project-p-bosxs1pdacsx/sagemaker-model-depl/SourceArti/8W7Xs2J
delete: s3://sagemaker-project-p-bosxs1pdacsx/sagemaker-model-depl/SourceArti/vHxe170
delete: s3://sagemaker-project-p-bosxs1pdacsx/sagemaker-model-depl/TestArtifa/mA3XwnD
delete: s3://sagemaker-project-p-bosxs1pdacsx/sagemaker-model-depl/TestArtifa/1tbEY7h
remove_bucket failed: s3://sagemaker-project-p-bosxs1pdacsx An error occurred (AccessDenied) when calling the DeleteBucket operation: Access Denied
delete: s3://sagemaker-project-p-xa5gxbybcnsa/from-idea-to-prod-pipeline/code/872b6ddc12ae2a57b03815cee5d86776/preprocessing.py
delete: s3://sagemaker-project-p-xa5gxbybcnsa/from-idea-to-prod-pipeline/code/bab9db759e3084d089c4a87f85d617ca/evaluation.py
delete: s3://sagemaker-project-p-xa5gxbybcnsa/from-idea-to-prod-pipeline/evaluate-2

## Remove inference endpoints

In [19]:
endpoints = sm.list_endpoints()["Endpoints"]

In [21]:
endpoints_to_delete = []

for ep in endpoints:
    print(f"Are you sure you want to delete this endpoint: {ep['EndpointName']}? (y/n)")
    choice = input()
    if choice == 'y':
        endpoints_to_delete.append(ep['EndpointName'])
        
print(f"*********** THESE ENDPOINTS WILL BE DELETED ***********")
print('\n'.join(endpoints_to_delete))
print(f"*******************************************************")

Are you sure you want to delete this endpoint: from-idea-to-prod-endpoint-17-15-21-23? (y/n)


 y


*********** THESE ENDPOINTS WILL BE DELETED ***********
from-idea-to-prod-endpoint-17-15-21-23
*******************************************************


In [22]:
for ep in endpoints_to_delete:
    try:
        delete_mon_schedules(ep, sm)
        time.sleep(10)
        print(f"Deleting the endpoint: {ep}:{sm.delete_endpoint(EndpointName=ep)}")
    except Exception as e:
        print(f"Exception in deleting {ep}:{e}")
        pass

Delete monitoring schedules for the endpoint: from-idea-to-prod-endpoint-17-15-21-23
Deleting the endpoint: from-idea-to-prod-endpoint-17-15-21-23:{'ResponseMetadata': {'RequestId': 'fa777508-a3d7-44c4-b910-f6cf331ad953', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'fa777508-a3d7-44c4-b910-f6cf331ad953', 'content-type': 'application/x-amz-json-1.1', 'content-length': '0', 'date': 'Sat, 18 Feb 2023 10:20:37 GMT'}, 'RetryAttempts': 0}}


## Remove project-related objects from the SageMaker S3 data bucket

<div class="alert alert-info"> 💡 <strong> The following code cells will delete all objects under specified S3 prefixes!</strong>
</div>

In [23]:
prefixes_to_delete = [
    bucket_prefix,
    #s3_fs_query_output_prefix
]

In [24]:
print(f"************************************************************************")
print(f"All objects under the following S3 prefixes will be removed permanently!")
print(f"************************************************************************")
for p in prefixes_to_delete:
    print(f"{bucket_name}/{p}")

************************************************************************
All objects under the following S3 prefixes will be removed permanently!
************************************************************************
sagemaker-us-east-1-462832133259/from-idea-to-prod/xgboost


<div class="alert alert-info"> ❗ <strong> Double check the S3 prefix! All data under this prefix will be permanently deleted!</strong>
</div>

In [26]:
# Un-comment the rm command
for p in prefixes_to_delete:
    # !aws s3 rm s3://{bucket_name}/{p} --recursive
    pass

# Shutdown kernel

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>